# Supplementary: End-to-end testing MagNet with DeepGlobe images

*ACCV'20 Submission - ID 323*

In this notebook, we will present end-to-end segmentation process with our MagNet framework on DeepGlobe sample images. MagNet is a fast and lightweight framework in high resolution image segmentation.

As reported in the paper, we use the Resnet-50 FPN as our backbone to test with 2448x2448 images in DeepGlobe testing set. To run this notebook, we prefer using [Google Colab](https://colab.research.google.com/) has all essential libraries installed for running experiments on GPU.

To open this notebook in Google Colab, click [here](https://colab.research.google.com/github/anonymous2020323/accv20-323/blob/master/deepglobe_testing.ipynb)

## Set up files

To run the experiments, you need to choose **Python 3** with **GPU** in notebook settings.

Then, run the following code to download essential files (trained models, sample images, and utilities)

In [ ]:
!wget https://storage.googleapis.com/anonymous-accv2020/segmentation.zip
!unzip segmentation.zip

## Import libraries

In [ ]:
import os

import cv2
import numpy as np
from PIL import Image

import torch

from magnet import MagNet
from utils import prediction2PIL, mIoU

## Constants for DeepGlobe dataset

In [ ]:
n_classes = 7                                                  # Number of classes
class2RGB = {                                                  # Map classID to color in the label image
    0: [0, 0, 0],
    1: [0, 255, 255],
    2: [255, 255, 0],
    3: [255, 0, 255],
    4: [0, 255, 0],
    5: [0, 0, 255],
    6: [255, 255, 255]
}

# MagNet config
n_levels = 3                                                   # Number of scale levels
backbone = "ResnetFPN"                                         # Backbone network
image_size = (2448, 2448)                                      # Original image size
patch_sizes = [(2448, 2448), (1224, 1224), (612, 612)]         # Patch size for each scale level
downsample_size = (508, 508)                                   # Size to downsample to
device = "cuda"

In [ ]:
images = [                                                              # image paths for testing
    "samples/deepglobe/606_sat.jpg",
    "samples/deepglobe/28935_sat.jpg",
    "samples/deepglobe/842556_sat.jpg",
]
labels = [image.replace("_sat.jpg", "_mask.png") for image in images]   # label paths for testing

In [ ]:
model_paths = [                                                         # model weight to restore
    "models/deepglobe/resnetfpn_level1_2448_508.pth",
    "models/deepglobe/resnetfpn_level2_1224_508.pth",
    "models/deepglobe/resnetfpn_level3_612_508.pth"
]

## Segment DeepGlobe images

### Prepare images and models

Create a MagNet instance for testing

In [ ]:
magnet = MagNet(backbone, n_levels, model_paths, device, n_classes, image_size, patch_sizes, downsample_size)

Choose image to test

In [ ]:
image_id = 0 # You can replace with 0,1,2

Load and show image (have been resize to show)

In [ ]:
# Read and show image
image = Image.open(images[image_id]).convert("RGB")
image.resize((512,512))

Segment images with 3 scales

### Segment image with MagNet

In [ ]:
pred = magnet.segment(image)

### Show results and mIoU of each scale level

#### Load and show segmentation ground-truth

In [ ]:
label_path = labels[image_id]
label = Image.open(label_path).convert("RGB")
label.resize((512,512))

#### For scale 1

Show the prediction

In [ ]:
pred_scale1 = prediction2PIL(class2RGB, pred[0])
pred_scale1.resize((512,512))

Calculate mIoU between scale 1 and label

In [ ]:
iou = mIoU(class2RGB, label, pred_scale1)
print("IoU at scale 1 is %.2f %%" % (iou*100))

#### For scale 2

The prediction

In [ ]:
pred_scale2 = prediction2PIL(class2RGB, pred[1])
pred_scale2.resize((512,512))

The IoU of scale 2

In [ ]:
iou = mIoU(class2RGB, label, pred_scale2)
print("IoU at scale 2 is %.2f %%" % (iou*100))

#### For scale 3

Show the prediction

In [ ]:
pred_scale3 = prediction2PIL(class2RGB, pred[2])
pred_scale3.resize((512,512))

The IoU at scale 3

In [ ]:
iou = mIoU(class2RGB, label, pred_scale3)
print("IoU at scale 3 is %.2f %%" % (iou*100))